In [42]:
import os
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
import pandas as pd 

from datasets import load_dataset
import voyageai

from langchain.vectorstores import Chroma
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain_experimental.text_splitter import SemanticChunker
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

from langchain_core.runnables import RunnableLambda
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field

In [2]:
with open("openai_api_key.txt", "r") as file:
    openai_key = file.read().strip()

with open("voyage_api_key.txt", "r") as file:
    voyage_api_key = file.read().strip()

os.environ["OPENAI_API_KEY"] = openai_key

## Dataset

In [3]:
# Load the dataset
dataset = load_dataset("ibm/finqa", trust_remote_code=True)

# Access the splits
data = dataset['train'].to_pandas()
validation_data = dataset['validation'].to_pandas()
test_data = dataset['test'].to_pandas()

data = pd.concat([data, validation_data, test_data])
data.reset_index(drop=True, inplace=True)
data = data[["id", "pre_text", "table", "post_text", "question", "answer"]]

In [4]:
data["Company"] = [row[0] for row in data.id.str.split("/")]
data["Year"] = [row[1] for row in data.id.str.split("/")]

In [5]:
table = list(data.iloc[0].table)

In [6]:
def json_to_markdown(json_data):
    # Create a DataFrame from the JSON data
    df = pd.DataFrame(json_data[1:], columns=json_data[0])

    # Convert the DataFrame to a Markdown table
    return df.to_markdown(index=False)

In [7]:
data["full_text"] = data.apply(
    lambda row: "\n".join(row.pre_text) + "\n\n\n\n" + json_to_markdown(list(row.table)) + "\n\n\n\n" + "\n".join(row.post_text),
    axis=1
)

data.drop(columns=["pre_text", "table", "post_text"], inplace=True)

In [8]:
print(data.full_text[1])

abiomed , inc .
and subsidiaries notes to consolidated financial statements 2014 ( continued ) note 8 .
stock award plans and stock-based compensation ( continued ) restricted stock and restricted stock units the following table summarizes restricted stock and restricted stock unit activity for the fiscal year ended march 31 , 2012 : number of shares ( in thousands ) weighted average grant date fair value ( per share ) .



|                                                                  | number of shares ( in thousands )   | weighted average grant date fair value ( per share )   |
|:-----------------------------------------------------------------|:------------------------------------|:-------------------------------------------------------|
| restricted stock and restricted stock units at beginning of year | 407                                 | $ 9.84                                                 |
| granted                                                          | 607        

In [63]:
query = data.iloc[0].question
data.iloc[0]

id                                      ADI/2009/page_49.pdf-1
question             what is the the interest expense in 2009?
answer                                                     380
Company                                                    ADI
Year                                                      2009
full_text    interest rate to a variable interest rate base...
Name: 0, dtype: object

## Indexing

In [9]:
text_splitter = RecursiveCharacterTextSplitter()

In [10]:
vo = voyageai.Client(api_key=voyage_api_key)

class Embedder:
    def __init__(self, batch_size=128):
        self.batch_size = batch_size  

    def embed_document(self, text):
        embedding = vo.embed([text], model="voyage-3", input_type="document").embeddings[0]
        return embedding

    def embed_documents(self, texts):
        embeddings = []
        for i in range(0, len(texts), self.batch_size):
            batch = texts[i:i + self.batch_size]
            batch_embeddings = vo.embed(batch, model="voyage-3", input_type="document").embeddings
            embeddings.extend([embedding for embedding in batch_embeddings])
        return embeddings
    
    def embed_query(self, query):
        embedding = vo.embed([query], model="voyage-3", input_type="query").embeddings[0]
        return embedding
    
embedder = Embedder()

In [11]:
from concurrent.futures import ThreadPoolExecutor, as_completed

persist_directory = ".chroma"

docs = []

def process_document(id, text):
    local_docs = []
    try:
        chunks = text_splitter.split_text(text)

        for i, chunk in enumerate(chunks):
            doc = Document(page_content=chunk, metadata={"id": data.loc[id].id, "chunk": i, "company": data.loc[id].Company, "year": data.loc[id].Year})
            local_docs.append(doc)

    except Exception as e:
        print(f"Error processing document {id}: {e}")
    
    return local_docs

if not os.path.exists(persist_directory):
    with ThreadPoolExecutor() as executor:
        futures = {executor.submit(process_document, id, text): id for id, text in data.full_text.iloc[0:10].items()}
        
        for future in as_completed(futures):
            result = future.result()
            docs.extend(result) 

In [12]:
from math import ceil

persist_directory = ".chroma"

if os.path.exists(persist_directory):

    # Load the existing ChromaDB
    chroma_db = Chroma(persist_directory=persist_directory, embedding_function=embedder)
    print("Loaded existing ChromaDB from .chroma")

else:

    # Create ChromaDB and store the documents
    chroma_db = Chroma(
        embedding_function=embedder,
        persist_directory=persist_directory,
    )
    
    print("Created new ChromaDB and saved to .chroma")

    batch_size = 5000
    num_batches = ceil(len(docs) / batch_size)

    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = min(start_idx + batch_size, len(docs))
        batch_docs = docs[start_idx:end_idx]
        
        chroma_db.add_texts(
            texts=[doc.page_content for doc in batch_docs],
            metadatas=[doc.metadata for doc in batch_docs]
        )

        print(f"Batch {i+1} of {num_batches} added to ChromaDB.")

/var/folders/mv/_dg3pqgn2zdf7f95_1dg07rw0000gn/T/ipykernel_57010/104589934.py:14: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  chroma_db = Chroma(


Created new ChromaDB and saved to .chroma
Batch 1 of 1 added to ChromaDB.


In [13]:
docs

[Document(metadata={'id': 'GIS/2019/page_45.pdf-1', 'chunk': 0, 'company': 'GIS', 'year': '2019'}, page_content='free cash flow conversion rate we believe this measure provides useful information to investors because it is important for assessing our efficiency in converting earnings to cash and returning cash to shareholders .\nthe calculation of free cash flow conversion rate and net cash provided by operating activities conversion rate , its equivalent gaap measure , follows: .\n\n\n\n| in millions                                                                                        | fiscal 2019      |\n|:---------------------------------------------------------------------------------------------------|:-----------------|\n| net earnings including earnings attributable to redeemable and noncontrolling interests asreported | $ 1786.2         |\n| net tax benefit ( a )                                                                              | $ -7.2 ( 7.2 )   |\n| tax item ( a 

## Retrieval

In [80]:
item = 2

query = data.iloc[item].question
answer = data.iloc[item].answer
company = data.iloc[item].Company
year = data.iloc[item].Year

retriever = chroma_db.as_retriever(search_kwargs={"k": 1, "filter": {"$and": [{"company": company}, {"year": year}]}})

## Generation

In [81]:
PROMPT = PromptTemplate(
    input_variables=["query", "context"],
    template="""
    Answer the following question based solely on the following context.

    Context: {context}

    Question: {query}
    """)

llm = ChatOpenAI()

In [82]:
def format_context(context):
    response = ""
    for doc in context:
        response += doc.page_content + "\n\n"
    return response

retrieve_chain = retriever | format_context 

generation_chain = RunnableLambda(lambda input: {
    "context": retrieve_chain.invoke(input["query"]),  # Call retriever correctly with a string input
    "query": input["query"]
}) | PROMPT | llm

In [83]:
print("query:", query)
print()

print(generation_chain.invoke(input={"query": query}).content)

print()
print("answer:", answer)

query: what was the total operating expenses in 2018 in millions

Total operating expenses in 2018 can be calculated by dividing the aircraft fuel expense (in millions) by the percent of total operating expenses. 

For 2018:
Aircraft fuel expense: $9896 million
Percent of total operating expenses: 23.6%

Total operating expenses in 2018:
$9896 million / 0.236 = $41,847.46 million

Therefore, the total operating expenses in 2018 was $41,847.46 million.

answer: 41932


In [84]:
print(data.iloc[item].full_text)

the following table shows annual aircraft fuel consumption and costs , including taxes , for our mainline and regional operations for 2018 , 2017 and 2016 ( gallons and aircraft fuel expense in millions ) .
year gallons average price per gallon aircraft fuel expense percent of total operating expenses .



|   year |   gallons | average priceper gallon   | aircraft fuelexpense   | percent of totaloperating expenses   |
|-------:|----------:|:--------------------------|:-----------------------|:-------------------------------------|
|   2018 |      4447 | $ 2.23                    | $ 9896                 | 23.6% ( 23.6 % )                     |
|   2017 |      4352 | 1.73                      | 7510                   | 19.6% ( 19.6 % )                     |
|   2016 |      4347 | 1.42                      | 6180                   | 17.6% ( 17.6 % )                     |



as of december 31 , 2018 , we did not have any fuel hedging contracts outstanding to hedge our fuel consumption .
